In [ ]:
!pip install sentence-transformers

In [3]:
from sentence_transformers import SentenceTransformer

In [67]:
# !pip install googletrans==4.0.0-rc1

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
import os
os.chdir("gdrive/MyDrive/GSN/churn-prediction-with-text-and-interpretability-main/scripts")

In [11]:
import yaml
import pandas as pd
from pathlib import Path

with open("../model/google/params.yaml", "r") as params_file:
    params_google = yaml.safe_load(params_file)
with open("../model/deepl/params.yaml", "r") as params_file:
    params_deepl = yaml.safe_load(params_file)

In [5]:
data_dir = params['data_dir']

df_text = pd.read_csv(Path(data_dir, "text.csv"))

In [6]:
df_text

,churn,chat_log
0,no,Customer: Text.\nTelCom Agent: What's the mini...
1,no,Customer: Just voice.\nTelCom Agent: And how m...
2,no,NaN
3,no,Customer: I can't upgrade my voice package I'm...
4,no,Customer: I'm looking to upgrade my contract t...
...,...,...
3328,no,"Customer: Hmm. Well, I am looking for a U-vers..."
3329,no,Customer: And I've just noticed that I can't c...
3330,no,Customer: All four.\nTelCom Agent: Okay. Can y...
3331,no,Customer: I would like to upgrade my data for ...


In [7]:
nan_index = df_text.chat_log.isna()

In [8]:
import numpy as np 
nan_index2 = np.invert(nan_index)

In [ ]:

list(df_text.chat_log.values[nan_index2])

In [18]:
from googletrans import Translator
translator = Translator()
translations = []
for i, chat in enumerate(df_text.chat_log.values[nan_index2]):
  if i%100==0:
    print(i)
  translations.append(translator.translate(chat, dest='pl'))

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600


In [19]:
import pickle

In [24]:
trans_google = [translation.text for translation in translations]

In [26]:
pickle.dump(trans_google, open('translations_google.pickle', 'wb'))

In [27]:
!pip install --upgrade deepl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
import deepl

auth_key = '...'  # Replace with your key
translator = deepl.Translator(auth_key)

result = translator.translate_text("Hello, world!", target_lang="FR")
print(result.text)  # "Bonjour, le monde !"

Bonjour, le monde !


In [35]:
result = translator.translate_text("Hello, world!", target_lang="PL")
print(result.text) 

Witaj, świecie!


In [36]:
translations2 = []
for i, chat in enumerate(df_text.chat_log.values[nan_index2]):
  if i%100==0:
    print(i)
  translations2.append(translator.translate_text(chat, target_lang="PL").text)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600


In [37]:
pickle.dump(translations2, open('translations_deepl.pickle', 'wb'))

In [41]:
df_text.head()

,churn,chat_log
0,no,Customer: Text.\nTelCom Agent: What's the mini...
1,no,Customer: Just voice.\nTelCom Agent: And how m...
2,no,NaN
3,no,Customer: I can't upgrade my voice package I'm...
4,no,Customer: I'm looking to upgrade my contract t...


In [46]:
df_text_google = pd.DataFrame(df_text['churn'].values, columns =['churn'])
df_text_deepl = pd.DataFrame(df_text['churn'].values, columns =['churn'])

In [51]:
df_text_deepl['chat_log']=None
df_text_deepl.loc[nan_index2,'chat_log'] = translations2


In [53]:
df_text_google['chat_log']=None
df_text_google.loc[nan_index2,'chat_log'] = trans_google

In [54]:
df_text_google.head()

,churn,chat_log
0,no,Klient: SMS.\nAgent Telcom: Jakie jest minimum...
1,no,Klient: Tylko głos.\nAgent Telcom: A ile to by...
2,no,None
3,no,Klient: Nie mogę zaktualizować pakietu głosowe...
4,no,Klient: Chcę ulepszyć umowę do Singtel.Jestem ...


In [57]:
df_text_google.to_csv('../data/df_text_google.csv', index=False)
df_text_deepl.to_csv('../data/df_text_deepl.csv', index=False)

In [55]:
df_text_deepl.head()

,churn,chat_log
0,no,Klient: Tekst.\nAgent TelCom: Jaka jest minima...
1,no,Klient: Tylko głos.\nAgent TelCom: A ile to bę...
2,no,None
3,no,Klient: Nie mogę uaktualnić mojego pakietu gło...
4,no,Klient: Chcę uaktualnić moją umowę z SingTel. ...


In [40]:

len(translations)

2636

In [39]:
len(translations2)

2636

Stwórz datasety

In [58]:
def create_joint_dataset(
    df_categorical, 
    df_text
):
    df_text_no = df_text[df_text.churn == 'no'].reset_index(drop=True)
    df_text_yes = df_text[df_text.churn == 'yes'].reset_index(drop=True)

    df_cat_no = df_categorical[df_categorical.churn == 'no'].reset_index(drop=True)[:len(df_text_no)]
    df_cat_yes = df_categorical[df_categorical.churn == 'yes'].reset_index(drop=True)[:len(df_text_yes)]

    df_no = pd.concat([df_text_no, df_cat_no.iloc[:,:-1]], axis=1)
    df_yes = pd.concat([df_text_yes, df_cat_yes.iloc[:,:-1]], axis=1)

    df = pd.concat([df_no, df_yes], axis=0)

    # shuffle data
    df = df.sample(frac=1).reset_index(drop=True)

    return df

In [59]:
data_dir = params['data_dir']
    
# load data
df_categorical = pd.read_csv(Path(data_dir, "train.csv"))
    
# create joint dataset
df_google = create_joint_dataset(df_categorical, df_text_google)
df_deepl = create_joint_dataset(df_categorical, df_text_deepl)
    
# save data
df_google.to_csv(Path(data_dir, "churn_dataset_google.csv"), index=False)
df_deepl.to_csv(Path(data_dir, "churn_dataset_deepl.csv"), index=False)

In [14]:
import os
import sys
import json
import yaml
import joblib
import logging
import argparse
import numpy as np
import pandas as pd
from pathlib import Path
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sentence_transformers import SentenceTransformer


class BertEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, model_name='dkleczek/bert-base-polish-uncased-v1'):
        self.model = SentenceTransformer(model_name)
        self.model.parallel_tokenization = False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        output = []
        for sample in X:
            encodings = self.model.encode(sample)
            output.append(encodings)
        return output


def extract_labels(
    data,
    label_name
):
    labels = []
    for sample in data:
        value = sample[label_name]
        labels.append(value)
    labels = np.array(labels).astype('int')

    return labels.reshape(labels.shape[0],-1)


def convert_label(
    df
):
    df.churn = df.churn.replace("no", 0)
    df.churn = df.churn.replace("yes", 1)
    return df

def extract_numerical_features(
    sample,
    numerical_feature_names
):
    output = []
    for feature_name in numerical_feature_names:
        if feature_name in sample.keys():
            value = sample[feature_name]
            if value is None:
                value = np.nan
        else:
            value = np.nan
        output.append(value)
    return output


def extract_categorical_features(
    sample,
    categorical_feature_names
):
    output = []
    for feature_name in categorical_feature_names:
        if feature_name in sample.keys():
            value = sample[feature_name]
            if value is None:
                value = ""
        else:
            value = ""
        output.append(value)
    return output


def extract_textual_features(
    sample,
    textual_feature_names
):
    output = []
    for feature_name in textual_feature_names:
        if feature_name in sample.keys():
            value = sample[feature_name]
            if value is None:
                value = ""
        else:
            value = ""
        output.append(value)
    return output


def split_data(
    df,
    label_name,
    test_size
):
    """Splits data and creates json format.
    """
    X = df.drop(columns=[label_name], axis=1)
    y = df[label_name]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=123, stratify=y)
    
    train = pd.DataFrame(X_train, columns = X.columns)
    train[label_name] = y_train
    test = pd.DataFrame(X_test, columns = X.columns)
    test[label_name] = y_test
    
    # create list of dicts
    train = train.to_json(orient="records")
    train = json.loads(train)
    test = test.to_json(orient="records")
    test = json.loads(test)
    
    return train, test


def extract_features(
    data,
    numerical_feature_names,
    categorical_feature_names,
    textual_feature_names
):
    """extract features by given feature names.
    """
    numerical_features = []
    categorical_features = []
    textual_features = []
    for sample in data:
        num_feat = extract_numerical_features(sample, numerical_feature_names)
        numerical_features.append(num_feat)
        cat_feat = extract_categorical_features(sample, categorical_feature_names)
        categorical_features.append(cat_feat)
        text_feat = extract_textual_features(sample, textual_feature_names)
        textual_features.append(text_feat)
        
    textual_features = [i if isinstance(i[0], str) else ["nan"] for i in textual_features]
    textual_features = [i[0] for i in textual_features]

    return numerical_features, categorical_features, textual_features


def save_feature_names(
    numerical_feature_names,
    categorical_feature_names,
    textual_feature_names,
    filepath
):
    feature_names = {
        'numerical': numerical_feature_names,
        'categorical': categorical_feature_names,
        'textual': textual_feature_names
    }
    with open(filepath, 'w') as f:
        json.dump(feature_names, f)


def load_feature_names(filepath):
    with open(filepath, 'r') as f:
        feature_names = json.load(f)
    numerical_feature_names = feature_names['numerical']
    categorical_feature_names = feature_names['categorical']
    textual_feature_names = feature_names['textual']
    return numerical_feature_names, categorical_feature_names, textual_feature_names


def get_feature_names(
    df
):
    num_columns = df.select_dtypes(include=np.number).columns.tolist()
    numerical_feature_names = [i for i in num_columns if i not in ['churn']]

    cat_columns = df.select_dtypes(include='object').columns.tolist()
    categorical_feature_names = [i for i in cat_columns if i not in ['chat_log']]

    textual_feature_names = ['chat_log']
    label_name = 'churn'

    return numerical_feature_names, categorical_feature_names, textual_feature_names, label_name
    

def prep_data(
    df, use_existing=False, test_size=0.33
):  
    """
    Args:
        df: Pandas dataframe with raw data
        use_existing: Set to True if you want to use locally stored, 
        already prepared train/test data. Set to False if you want 
        to rerun the data preparation pipeline.
    Returns:
        Train and test data as well as train labels and test labels.
    """
    train_file = Path(model_dir, 'train.csv')
    labels_file = Path(model_dir, 'labels.csv')
    test_file = Path(model_dir, 'test.csv')
    labels_test_file = Path(model_dir, 'labels_test.csv')
    feature_names_file = Path(model_dir, "feature_names.json")
    oh_feature_names_file = Path(model_dir, "one_hot_feature_names.json")
    all_file_paths = [train_file, labels_file, test_file, labels_test_file,
                     feature_names_file, oh_feature_names_file]
    
    if use_existing == True and all(file.exists() for file in all_file_paths):
        features = np.array(pd.read_csv('../model/train.csv'))
        labels = np.array(pd.read_csv('../model/labels.csv'))
        features_test = np.array(pd.read_csv('../model/test.csv'))
        labels_test = np.array(pd.read_csv('../model/labels_test.csv'))
        print("Using already prepared data.")

    else:
        print("Running data preparation pipeline...")
        # convert label to binary numeric
        df = convert_label(df)

        # extract feature names
        numerical_feature_names, categorical_feature_names, textual_feature_names, label_name = get_feature_names(
            df
        )
        # train/test split and convert to json format (list of dicts)
        train, test = split_data(
            df,
            label_name,
            test_size
        )
        # extract features & label
        print('extracting features')
        numerical_features, categorical_features, textual_features = extract_features(
            train,
            numerical_feature_names,
            categorical_feature_names,
            textual_feature_names
        )
        labels = extract_labels(
            train,
            label_name
        )
        # extract features & label (for test data)
        numerical_features_test, categorical_features_test, textual_features_test = extract_features(
            test,
            numerical_feature_names,
            categorical_feature_names,
            textual_feature_names
        )
        labels_test = extract_labels(
            test,
            label_name
        )
        # define preprocessors
        print('defining preprocessors')
        numerical_transformer = SimpleImputer(missing_values=np.nan, strategy='mean', add_indicator=True)
        categorical_transformer = OneHotEncoder(handle_unknown="ignore")
        textual_transformer = BertEncoder()

        # fit preprocessors
        print('fitting numerical_transformer')
        numerical_transformer.fit(numerical_features)
        print('saving numerical_transformer')
        joblib.dump(numerical_transformer, Path(model_dir, "numerical_transformer.joblib"))
        print('fitting categorical_transformer')
        categorical_transformer.fit(categorical_features)
        print('saving categorical_transformer')
        joblib.dump(categorical_transformer, Path(model_dir, "categorical_transformer.joblib"))

        # transform features
        print('transforming numerical_features')
        numerical_features = numerical_transformer.transform(numerical_features)
        print('transforming categorical_features')
        categorical_features = categorical_transformer.transform(categorical_features)
        print('transforming textual_features')
        textual_features = textual_transformer.transform(textual_features)

        # transform features (for test data)
        print('transforming numerical_features_test')
        numerical_features_test = numerical_transformer.transform(numerical_features_test)
        print('transforming categorical_features_test')
        categorical_features_test = categorical_transformer.transform(categorical_features_test)
        print('transforming textual_features_test')
        textual_features_test = textual_transformer.transform(textual_features_test)

        # concat features
        print('concatenating features')
        categorical_features = categorical_features.toarray()
        textual_features = np.array(textual_features)
        textual_features = textual_features.reshape(textual_features.shape[0], -1)
        features = np.concatenate([
            numerical_features,
            categorical_features,
            textual_features
        ], axis=1)

        # concat features (test data)
        print('concatenating features of test data')
        categorical_features_test = categorical_features_test.toarray()
        textual_features_test = np.array(textual_features_test)
        textual_features_test = textual_features_test.reshape(textual_features_test.shape[0], -1)
        features_test = np.concatenate([
            numerical_features_test,
            categorical_features_test,
            textual_features_test
        ], axis=1)

        # save to disk
        pd.DataFrame(features).to_csv(Path(model_dir, "train.csv"), index=False)
        pd.DataFrame(labels).to_csv(Path(model_dir, "labels.csv"), index=False)
        pd.DataFrame(features_test).to_csv(Path(model_dir, "test.csv"), index=False)
        pd.DataFrame(labels_test).to_csv(Path(model_dir, "labels_test.csv"), index=False)

        save_feature_names(
            numerical_feature_names,
            categorical_feature_names,
            textual_feature_names,
            Path(model_dir, "feature_names.json")
        )
        # one-hot encoded feature names (for feat_imp)
        save_feature_names(
            numerical_feature_names,
            categorical_transformer.get_feature_names().tolist(),
            textual_feature_names,
            Path(model_dir, "one_hot_feature_names.json")
        )

    return features, features_test, labels, labels_test

In [71]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("dkleczek/bert-base-polish-uncased-v1")
model = AutoModelForMaskedLM.from_pretrained("dkleczek/bert-base-polish-uncased-v1")

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/507M [00:00<?, ?B/s]

Some weights of the model checkpoint at dkleczek/bert-base-polish-uncased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Googlowe tłumaczenie

In [15]:
data_dir = params_google['data_dir']
model_dir = params_google['model_dir']
df_google = pd.read_csv(Path(data_dir, "churn_dataset_google.csv"))
X_train, X_test, y_train, y_test = prep_data(df_google, use_existing=False, test_size=0.33)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

Running data preparation pipeline...
extracting features
defining preprocessors


Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.75k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/531M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/dkleczek_bert-base-polish-uncased-v1 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


fitting numerical_transformer
saving numerical_transformer
fitting categorical_transformer
saving categorical_transformer
transforming numerical_features
transforming categorical_features
transforming textual_features
transforming numerical_features_test
transforming categorical_features_test
transforming textual_features_test
concatenating features
concatenating features of test data


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


((2233, 841), (1100, 841), (2233, 1), (1100, 1))

In [70]:
X_train[0]

array([ 9.30000000e+01,  2.10000000e+01,  1.34200000e+02,  1.05000000e+02,
        2.28100000e+01,  1.62500000e+02,  1.28000000e+02,  1.38100000e+01,
        1.86600000e+02,  9.00000000e+01,  8.40000000e+00,  1.18000000e+01,
        2.00000000e+00,  3.19000000e+00,  4.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  

In [17]:
import train

../model
../model


In [79]:
train.train(
    X=X_train,
    y=y_train,
    X_test=X_test,
    y_test=y_test
)

starting epoch:  1


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Train Epoch: 1, train-auc-score: 0.9325


INFO:train:Train Epoch: 1, train-auc-score: 0.9325
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


test_auc_score: 0.9177


...

Train Epoch: 10, train-auc-score: 0.9826


INFO:train:Train Epoch: 10, train-auc-score: 0.9826
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


test_auc_score: 0.9749


INFO:train:test_auc_score: 0.9749


saving scores
saving model


In [75]:
model_dir = params['model_dir']

In [76]:
scores_df = pd.read_csv(Path(model_dir, 'training_scores.csv'))

In [77]:
scores_df

,train_scores,test_scores
0,0.941793,0.925745
1,0.953943,0.942534
2,0.926605,0.921681
3,0.962444,0.955286
4,0.971268,0.960279
5,0.971105,0.963140
6,0.962740,0.948128
7,0.968171,0.963039
8,0.942372,0.939339
9,0.978338,0.970144


# Deepl tlumaczenie

In [80]:
with open("../model/params.yaml", "r") as params_file:
    params = yaml.safe_load(params_file)

model_dir = params['model_dir']

In [81]:
model_dir

'../model/deepl'

In [82]:
import preprocess

In [83]:
X_train, X_test, y_train, y_test = preprocess.prep_data(df_deepl, use_existing=False, test_size=0.33)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

Running data preparation pipeline...
extracting features
defining preprocessors


Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/dkleczek_bert-base-polish-uncased-v1 were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


fitting numerical_transformer
saving numerical_transformer
fitting categorical_transformer
saving categorical_transformer
transforming numerical_features
transforming categorical_features
transforming textual_features
transforming numerical_features_test
transforming categorical_features_test
transforming textual_features_test
concatenating features
concatenating features of test data


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


((2233, 841), (1100, 841), (2233, 1), (1100, 1))

In [84]:
import train

In [85]:
train.train(
    X=X_train,
    y=y_train,
    X_test=X_test,
    y_test=y_test
)

starting epoch:  1


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Train Epoch: 1, train-auc-score: 0.9304


INFO:train:Train Epoch: 1, train-auc-score: 0.9304
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


test_auc_score: 0.9452


...

Train Epoch: 10, train-auc-score: 0.9802


INFO:train:Train Epoch: 10, train-auc-score: 0.9802
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


test_auc_score: 0.9695


INFO:train:test_auc_score: 0.9695


saving scores
saving model


In [18]:
model_dir = params_deepl['model_dir']
scores_df = pd.read_csv(Path('../model', 'training_scores.csv'))
scores_df

,train_scores,test_scores
0,0.930372,0.945207
1,0.944905,0.949131
2,0.961854,0.961128
3,0.941484,0.939526
4,0.971107,0.965713
5,0.967991,0.956657
6,0.974647,0.965158
7,0.969503,0.956971
8,0.978988,0.965947
9,0.980212,0.969462
